In [1]:
import json 
import pandas as pd
import numpy as np
import uuid
import re
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS, PROV, SKOS, RDFS
from functions import *
from namespaces import *
from shapely.geometry import shape

In [2]:
#import json files
with open('/workspaces/ontologie-peuplement/data/gentilly/parcelles_1811.geojson') as f:
    data_1811 = json.load(f)

with open('/workspaces/ontologie-peuplement/data/gentilly/parcelles_1848.geojson') as f:
    data_1845 = json.load(f)

In [3]:
data_1845

{'type': 'FeatureCollection',
 'name': 'parcelles_1848',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::2154'}},
 'features': [{'type': 'Feature',
   'properties': {'fid': 1, 'numero': '1', 'section': 'D', 'nature': 'NB'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[652493.0942947898, 6859103.821806228],
      [652518.1291845728, 6859114.864593228],
      [652528.4860841817, 6859091.270066966],
      [652498.1698614856, 6859078.238206532],
      [652498.1012727465, 6859082.353530879],
      [652493.6430047029, 6859080.981756097],
      [652493.0942947898, 6859103.821806228]]]}},
  {'type': 'Feature',
   'properties': {'fid': 2, 'numero': '2', 'section': 'D', 'nature': 'NB-B'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[652518.1291845728, 6859114.864593228],
      [652532.7385860079, 6859121.174757228],
      [652543.1640743559, 6859097.785997184],
      [652528.4860841817, 6859091.270066966],
      [652518.1291845728, 6859114.864593228

In [4]:
COMMUNE = 'Gentilly'
matrices_metada = {
    "MAT_1813": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_B_NB_1813",
        "MATRICE_START": "1813",
        "MATRICE_END": "1835"
    },
    "MAT_1836": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_NB_1836",
        "MATRICE_START": "1836",
        "MATRICE_END": "1847"
    },
    "MAT_1848": {
        "PLAN": "1845",
        "MATRICE_ID": "MAT_NB_1848",
        "MATRICE_START": "1848",
        "MATRICE_END": "1860"
    }
}

In [8]:
#Créer un objet Turtle avec une URI
g = Graph()

g.bind('landmark', landmarkuri)
g.bind('owner', owneruri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)
g.bind('cad_atype', cad_atype)
g.bind('pnature',pnature)
g.bind('ctype',ctype)
g.bind('atype',atype)

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

In [9]:
def get_base_Skos_Concept(str):
    if str == 'B':
        return 'ProprieteBatie'
    elif str == 'NB':
        return 'ProprieteNonBatie'
    elif str == 'B-NB' or str == 'NB-B':
        return 'ProprieteBatie;ProprieteNonBatie'
    
def geojson_to_wkt(geojson_geometry):
    shapely_geometry = shape(geojson_geometry)
    return shapely_geometry.wkt

In [10]:
for elem in data_1811['features']:
    resource_uri = URIRef(landmarkuri + str(uuid.uuid4()))
    g.add((resource_uri, RDF.type, add.Landmark))
    g.add((resource_uri, add.isLandmarkType, cad_ltype.Plot))
    g.add((resource_uri, DCTERMS.identifier, Literal(elem['properties']['section'] + '-' + elem['properties']['numero'], datatype=XSD.string)))

    #Plot Base Landmark Relation : fix Landmark Relation of the plot : Plot => Section => Commune
    landmarkRelation = BNode()
    g.add((URIRef(landmarkRelation.n3()), RDF.type, add.LandmarkRelation))
    g.add((URIRef(landmarkRelation.n3()), add.locatum, resource_uri))
    g.add((URIRef(landmarkRelation.n3()), add.relatum, URIRef(landmarkuri + "87d7c2f6-306b-45a1-a833-5e17821c3102")))

    #Plot Address : relative to thorofares or districts mentionned in the registers

    #Landmark Appearance
    ## Change
    changeapp = BNode()
    g.add((URIRef(changeapp.n3()), RDF.type, add.Change))
    g.add((URIRef(changeapp.n3()),add.appliedTo,resource_uri))
    g.add((URIRef(changeapp.n3()),add.isChangeType,ctype.LandmarkAppearance))
    
    ## Event
    eventapp = BNode()
    g.add((URIRef(eventapp.n3()), RDF.type, add.Event))
    g.add((URIRef(changeapp.n3()),add.dependsOn,URIRef(eventapp.n3())))
    time_ = BNode()
    g.add((URIRef(eventapp.n3()), add.hasTime, time_))
    g.add((time_, RDF.type, add.TimeInstant))
    g.add((time_, add.timeCalendar,time.Gregorian))
    g.add((time_, add.timePrecision,time.Year))
    g.add((time_, add.timeValue, Literal('1811', datatype=XSD.dateTimeStamp)))

    #Landmark Disappearance
    ## Change
    changedis = BNode()
    g.add((URIRef(changedis.n3()), RDF.type, add.Change))
    g.add((URIRef(changedis.n3()),add.appliedTo,resource_uri))
    g.add((URIRef(changedis.n3()),add.isChangeType,ctype.LandmarkDisappearance))
    
    ## Event
    eventdis= BNode()
    g.add((URIRef(eventdis.n3()), RDF.type, add.Event))
    g.add((URIRef(changedis.n3()),add.dependsOn,URIRef(eventdis.n3())))

    # Plot Nature
    ## Version
    natureattr = BNode()
    g.add((natureattr,RDF.type,add.Attribute))
    g.add((resource_uri, add.hasAttribute, natureattr))
    g.add((natureattr, add.isAttributeType, cad_atype.PlotNature))
    natureattrversion = BNode()
    g.add((natureattr, add.hasAttributeVersion,natureattrversion))
    g.add((natureattrversion,RDF.type,add.AttributeVersion))
    nature = get_base_Skos_Concept(elem['properties']['nature'])
    if ';' in nature:
        for n in nature.split(';'):
            g.add((natureattrversion,add.hasPlotNature,URIRef(pnature+n)))
    else:
        g.add((natureattrversion,add.hasPlotNature,URIRef(pnature+nature)))
    
    ## Plot Nature Appearance
    ### Change
    changenatapp = BNode()
    g.add((URIRef(changenatapp.n3()), RDF.type, add.Change))
    g.add((URIRef(changenatapp.n3()),add.appliedTo,natureattr))
    g.add((URIRef(changenatapp.n3()),add.makesEffective,natureattrversion))
    ### Event
    eventnatapp = BNode()
    g.add((URIRef(eventnatapp.n3()), RDF.type, add.Event))
    g.add((URIRef(changenatapp.n3()),add.dependsOn,URIRef(eventnatapp.n3())))
    time_ = BNode()
    g.add((URIRef(eventnatapp.n3()), add.hasTime, time_))
    g.add((time_, RDF.type, add.TimeInstant))
    g.add((time_, add.timeCalendar,time.Gregorian))
    g.add((time_, add.timePrecision,time.Year))
    g.add((time_, add.timeValue, Literal('1811', datatype=XSD.dateTimeStamp)))

    ## Plot Nature Disappearance
    ### Change
    changenatdis = BNode()
    g.add((URIRef(changenatdis.n3()), RDF.type, add.Change))
    g.add((URIRef(changenatdis.n3()),add.appliedTo,natureattr))
    g.add((URIRef(changenatdis.n3()),add.outdates,natureattrversion))
    ### Event
    eventnatdis = BNode()
    g.add((URIRef(eventnatdis.n3()), RDF.type, add.Event))
    g.add((URIRef(changenatdis.n3()),add.dependsOn,URIRef(eventnatdis.n3())))

    #Plot Geometry
    geom = BNode()
    g.add((geom, RDF.type, add.Attribute))
    g.add((geom, add.isAttributeType, atype.Geometry))
    g.add((resource_uri, add.hasAttribute, geom))
    geomversion = BNode()
    g.add((geom, add.hasAttributeVersion, geomversion))
    g.add((geomversion, RDF.type, add.AttributeVersion))
    g.add((geomversion, add.versionValue, Literal(geojson_to_wkt(elem['geometry']) + "<http://www.opengis.net/def/crs/EPSG/0/2154>", datatype=geo.wktLiteral)))

print(g.serialize(format='turtle'))
g.serialize(destination=f"/workspaces/ontologie-peuplement/data/rdf/{COMMUNE}_landmarks_initial_plots.ttl", format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/> .
@prefix cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/> .
@prefix cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/> .
@prefix ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix landmark: <http://data.ign.fr/id/landmark/> .
@prefix pnature: <http://data.ign.fr/id/codes/cadastre/plotNature/> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<_:N0ad7d4f2b8a745d8be9d615bd30724fa> a add:Change ;
    add:appliedTo _:Nfbdcca6ed72e438f8b98613114876c4a ;
    add:dependsOn <_:Nce5e0c1723a54684865a5a95942870b3> ;
    add:makesEffective _:N9f05847d8fb340ca89887cc472c54638 .

<_:N0cbe5ad50762451ab036b8a4a84094cd> a add:Change ;
    add:appliedT

<Graph identifier=N5ff66c2518d045d08f834ac24b56fcf0 (<class 'rdflib.graph.Graph'>)>

Ajouter ce Turtle dans un graphe nommé "cad1/initialplots"